In [1]:
#from __future__ import unicode_literals
import json
from pprint import pprint
class FileLoader:
    """FileLoader takes as input an evenSet, era, and channel; it returns a list of files, a json, and a configuration dictionary.

eventSet: "TTTT", "TT", "DY", etc.
era: 2016, 2017, 2018. This corresponds to both MC tunes and Data run sets, as well as the default configuration parameters.
channel: SL or DL to return the appropriate configuration parameters that is loaded. Overriden by configName option.
configName: (Optional) When specified, returns the requested configuration instead of the standard one for that era and channel.
serverPrefix: the default prefix for files is root://cms-xrd-global.cern.ch/. Pass any other as a string in the same format. """

    def __init__(self, eventSet=None, era=None, channel=None, configName=None, jsonName=None, filePrefix="root://cms-xrd-global.cern.ch/"):
        #internal variables grabbed, except jsonName
        self._eventSet = eventSet
        self._era = era
        self._channel = channel
        self._configName = configName
        self._filePrefix = filePrefix
        
        #Make these all named options, but still require the first three to always be passed
        if (self._eventSet == None or self._era == None or self._channel == None):
            raise RuntimeError("FileLoader requires an eventSet, an era(2016, 2017, 2018), and an analysis channel(SL, DL)")

        #########################################
        ### USER DEFINED DATASETS and PATHS   ###
        ### Use convenient name and era       ###
        ### and finally indicate path inside  ###
        ### self._filePath defined first      ###
        ### Data must being with "Run"        ###
        ### Monte Carlo must NOT              ###
        #########################################
        self._jsonPath = "../json/"
        self._configPath = "../config/"
        self._filePath = "../filelists/"
        self._eventDict = { 
            "TTTT-PSweights" : { "2016" : "NOT IMPLEMENTED",
                                 "2017" : "TTTT_TuneCP5_PSweights_13TeV-amcatnlo-pythia8.txt",
                                 "2018" : "NOT IMPLEMENTED" },
            "TTTT" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "TTTT_TuneCP5_13TeV-amcatnlo-pythia8.txt",
                        "2018" : "NOT IMPLEMENTED" },
            "TTJetsSL" :  { "2016" : "NOT IMPLEMENTED",
                            "2017" : "TTJets_SingleLeptFromT_TuneCP5_13TeV-madgraphMLM-pythia8.txt",
                            "2018" : "NOT IMPLEMENTED" },
            "WJetsToLNu" :  { "2016" : "NOT IMPLEMENTED",
                              "2017" : "WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.txt",
                              "2018" : "NOT IMPLEMENTED" },
            "RunB" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunC" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunD" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunE" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunF" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunG" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" },
            "RunH" :  { "2016" : "NOT IMPLEMENTED",
                        "2017" : "NOT IMPLEMENTED",
                        "2018" : "NOT IMPLEMENTED" }
            }

        ######################################################
        ### Name all necessary inputs for convenience here ###
        ######################################################
        #Name configuration, using method that makes it convenient to add many without modifying this function
        self._configName = self._configPath + channel + "_" + era
        if configName is None:
            self._configName += "_default.json"
        else:
            self._configName += configName
        if self._configName[-5:] != ".json":
            self._configName += ".json"
        #Name filelist input
        self._filelistName = self._filePath + self._eventDict[eventSet][era]
        #Grab jsonName from input
        self._jsonName = jsonName
        
        #################################
        ### Set up ToReturn variables ###
        #################################
        #name event JSON if Data and no jsonName was passed in
        if eventSet[:3] is "Run":
            if self._jsonName is None:
                self._jsonToReturn = self._jsonPath + "Cert_294927-306462_13TeV_EOY2017ReReco_Collisions17_JSON.txt"
            else:
                self.jsonToReturn = self._jsonPath + self._jsonName
        else:
            self._jsonToReturn = None
        #Empty filelistToReturn
        self._filelistToReturn = []
        #Empty config file
        self._configToReturn = {}
        
        #############################################
        ### hasLoaded boolean for invoking load() ###
        #############################################
        self._hasLoaded = False

    def __load__(self):
        #Open config file in read-only mode, then load the json
        #try: #is this necessary, or implemented in "with" syntax automatically?
        with open(self._configName, "r") as inConfig:
            self._configToReturn = json.load(inConfig)
        #finally:
        #    inConfig.close()

        #Test that the JSON file can be opened, then do just close it
        if self._jsonToReturn is not None:
            try:
                f = open(self._jsonToReturn, "r")
            finally:
                f.close()
            
        with open(self._filelistName, "r") as inFiles:
            for line in inFiles:
                self._filelistToReturn.append(self._filePrefix + str(line).replace('\n',''))
            
        self._hasLoaded = True
        
    def printEventSets(self):
        pprint(self._eventDict)

    def printConfig(self):
        if self._hasLoaded:
            pprint(self._configToReturn)
        else:
            print("A configuration has yet to be loaded. Invoke getConfig() first.")

    def getFiles(self, indexOfFile=-1):
        if not self._hasLoaded:
            self.__load__()
        if indexOfFile < 0:
            return self._filelistToReturn
        elif indexOfFile > len(self._filelistToReturn):
            raise RuntimeError("You've requested a file that is beyond the index range available, which is 0 to {0:s}"
                               .format(self._filelistToReturn))
        else:
            return [self._filelistToReturn[indexOfFile]]

    def getConfig(self):
        if not self._hasLoaded:
            self.__load__()
        return self._configToReturn

    def getJSONpath(self):
        if not self._hasLoaded:
            self.__load__()
        return self._jsonToReturn

#Implement as class, create functions that return possible evensets, store internally so that these values to be returned (list of files, json) are always available through submethods...

In [ ]:
fl = FileLoader(eventSet="TTTT", era="2017", channel="DL")

In [ ]:
fl.printEventSets()

In [ ]:
fl.printConfig()

In [ ]:
y = fl.getConfig()

In [ ]:
z = fl.getFiles()

In [ ]:
print(z[0])

In [ ]:
fl2 = FileLoader(eventSet="RunB", era="2017", channel="DL")

In [ ]:
fl2.getFiles()

In [2]:
from FourTopNAOD.Prototype.modules.fileloader import FileLoader as IFL

ImportError: No module named FourTopNAOD.Prototype.modules.fileloader

In [ ]:
w = IFL(eventSet="TTTT", era="2017", channel="SL")

In [ ]:
w.getConfig()

In [1]:
from __future__ import (division, print_function)
import os, sys
import ROOT
ROOT.PyConfig.IgnoreCommandLineOptions = True

from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor
from FourTopNAOD.Prototype.modules.eventselector import ProtoEventSelector
from FourTopNAOD.Prototype.tools.fileloader import FileLoader as IFL

Welcome to JupyROOT 6.10/09


In [2]:
preselection="nJet > 4 && (nMuon + nElectron) > 1"

In [11]:
Y = 0
Y = IFL(eventSet="TTTT", era="2017", channel="DL")

In [12]:
print(Y.getFiles())
print(Y.getJSONPath())
print(Y.getConfig())
print(Y.getSet())

['root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/C6AABB0E-33AC-E811-8B63-0CC47A7C3404.root', 'root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/B0F839D7-45AC-E811-94F8-1418776375C9.root', 'root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/74E3A905-68AB-E811-8183-246E96D10B14.root', 'root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/4A01C126-89A9-E811-9393-0CC47AFC3D32.root', 'root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/0E1A857B-45AC-E811-8122-0025905A6

In [13]:
print(Y.getFiles(indexOfFile=0))

['root://cms-xrd-global.cern.ch//store/mc/RunIIFall17NanoAOD/TTTT_TuneCP5_13TeV-amcatnlo-pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/70000/C6AABB0E-33AC-E811-8B63-0CC47A7C3404.root']


In [17]:
p=PostProcessor(".",
                Y.getFiles(indexOfFile=0),
                cut=preselection,
                branchsel=None,
                outputbranchsel=None,
                postfix="_" + Y.getSet() + "_PostEvtSel",
                jsonInput=Y.getJSONPath(),
#                jsonInput={1 : [[10000, 19000]]},
                modules=[EventSelector(makeHistos=True, cutOnTrigs=True, cutOnMET=True, 
                                       cutOnHT=True, verbose=False, selectionConfig=Y.getConfig())],
#                modules=[showyEventSelector()],
                justcount=False,
                provenance=True,
                fwkJobReport=True,
                haddFileName="hadded.root",
                noOut=False,
                histFileName="histOut.root",
                histDirName="plots",
                compression="LZMA:9"
                )

In [18]:
p.modules[0].maxEventsToProcess = 1000
print(p.modules[0].maxEventsToProcess)

1000


In [16]:
p.run()

Will write selected trees to .
Pre-select 166448 entries out of 186826 
Processed    10000/  166448 entries (elapsed time     1.1s, curr speed    8.850 kHz, avg speed    8.850 kHz), accepted        4/   10001 events ( 0.04%)
Processed    20000/  166448 entries (elapsed time     1.2s, curr speed  166.667 kHz, avg speed   16.807 kHz), accepted        4/   20001 events ( 0.02%)
Processed    30000/  166448 entries (elapsed time     1.3s, curr speed  142.857 kHz, avg speed   23.810 kHz), accepted        4/   30001 events ( 0.01%)
Processed    40000/  166448 entries (elapsed time     1.3s, curr speed  142.857 kHz, avg speed   30.075 kHz), accepted        4/   40001 events ( 0.01%)
Processed    50000/  166448 entries (elapsed time     1.4s, curr speed  166.667 kHz, avg speed   35.971 kHz), accepted        4/   50001 events ( 0.01%)
Processed    60000/  166448 entries (elapsed time     1.5s, curr speed  142.857 kHz, avg speed   41.096 kHz), accepted        4/   60001 events ( 0.01%)
Processed 

Error in <TGClient::TGClient>: can't open display "localhost:20.0", switching to batch mode...
 In case you run from a remote ssh session, reconnect with ssh -Y
